In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
indexing_tables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [2]:
tf.__version__

'2.0.0-rc0'

In [3]:
raw_articles = pd.read_csv('../dataset/new_york_dataset/article_data_newyork', delimiter=',', header=None)
data_articles = raw_articles.iloc[:,:].values

In [4]:
# embedding_model = gs.models.FastText.load_fasttext_format('../pre_trained_models/cc.en.300.bin')
embedding_model = gs.models.FastText.load('../train_embedding_models/fasttext_embedding_50d_all_signals')

In [5]:
indexing_tables.indices.close(index='data_table_newyork')
indexing_tables.indices.put_settings(index='data_table_newyork', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexing_tables.indices.open(index='data_table_newyork')

{'acknowledged': True, 'shards_acknowledged': True}

In [33]:
def search_indexing(query):
    
    result= indexing_tables.search(
        index="data_table_newyork", 
        body = {
        "_source": ["table_url","table_page_title"],
        "from" : 0,
        "size" : 70,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["table_page_title","table_page_content"] 
            }
        }
    })
    
    return result

In [7]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID == ID_goal:
    
            accuracy = 1
            break;
            
    return accuracy

In [8]:
MAX_PAD = 31

def sequence_padding(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [9]:
def create_embedding(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD:
        
        embedding = embedding_model.wv[value]
        
        padding_embedding = sequence_padding(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD]]
        
        return embedding

In [10]:
def search_index(article_title):
    
    tables_index = []

    result_index = search_indexing(article_title)
        
    for hit in result_index['hits']['hits']:
    
        table_ID = hit['_source']['table_url']
        
        table_page_title = hit['_source']['table_page_title']
    
        tables_index.append([table_ID,table_page_title])
    
    return tables_index

In [32]:
ranking_model = tf.keras.models.load_model('best_model_title_affinity_1_1_08_0.9499.h5')

In [34]:
result = []

def run_search(k):
    
    TOP_K = k
    accuracy = []
    
    for row_article in tqdm(data_articles):
    
        article_title = []
        title_table = []
        ranked_tables_model = []
    
        article_ID = row_article[3]
        article_title_text = str(row_article[1])
        article_main_passage_text = str(row_article[2][0:2000])
        
        
        ranked_tables_index = search_index(article_title_text+" "+article_main_passage_text)
        
        article_title_embedding = create_embedding(article_title_text)
        
        for table_ID, table_title in (ranked_tables_index):
            
            table_title_embedding = create_embedding(str(table_title))
            
            article_title.append(article_title_embedding)
            title_table.append(table_title_embedding)
    
        article_title = np.array(article_title)
        title_table = np.array(title_table)
    
        table_ranking_model = ranking_model.predict([article_title,title_table])
    
        for i in range(0,len(table_ranking_model)):
        
            ranked_tables_model.append([ranked_tables_index[i][0],ranked_tables_index[i][1],table_ranking_model[i][0]]) 
        
        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking', ascending=False)
        final_ranked_tables = data_frame_sorting.iloc[0:TOP_K,0:1].values
        
#         selected_top = data_frame_sorting.head(TOP_K)
#         min_score = selected_top['table_ranking'].min()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] >= min_score]
#         final_ranked_tables = draw_tables_socres.iloc[:,0:1].values
           
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))
        
    result.append(["Acc@"+str(k),str(round(np.mean(accuracy),4))])
    #print("")
    #print("Acc@"+str(k))
    #print(str(round(np.mean(accuracy),4)))
    #print(accuracy)

In [35]:
accuracy_K = [1,5,10]

for k in accuracy_K:
     
    run_search(k)

100%|██████████| 148/148 [00:11<00:00, 13.52it/s]


In [36]:
result

[['Acc@1', '0.3649'], ['Acc@5', '0.4932'], ['Acc@10', '0.5473']]

In [ ]:
# affinity
# [['Acc@1', '0.3716'], ['Acc@5', '0.5135'], ['Acc@10', '0.5743']]

In [ ]:
## attention
## [['Acc@1', '0.3311'], ['Acc@5', '0.473'], ['Acc@10', '0.5541']]

In [ ]:
##[['Acc@1', '0.277'], ['Acc@5', '0.4324'], ['Acc@10', '0.5338']]